In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import multiprocessing as mp
import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib as mpl
import warnings

from cycler import cycler
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import StdDevUncertainty
from astropy.convolution import convolve
from glob import glob
from lmfit import Model
from lmfit.printfuncs import report_fit
from pathlib import Path
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from specutils.manipulation import LinearInterpolatedResampler
from specutils import Spectrum1D

#for warnings
from uncertainties import UFloat
from astropy.utils.exceptions import AstropyDeprecationWarning

### Plott formatting

In [442]:
clrs = sns.color_palette('Set1')
clrs[5] = (255/255, 200/255, 80/255)
colors = cycler('color', list(clrs.as_hex()))
mpl.rc('font', family='serif')
mpl.rc('text', usetex=True)

plt.rc('axes', axisbelow=True, 
       grid=False, prop_cycle=colors)
plt.rc('xtick', direction='in')
plt.rc('ytick', direction='in')
plt.rcParams['figure.dpi'] = 75

In [443]:
# monte-carlo error estimation
# def bootstrap_err(self, x, model, flux, fluxerr, params, num_boots, method='leastsq'):
    #     paramNames = model.param_names
    #     param_kwargs = {name: params[name] for name in paramNames}
        
    #     # Preallocate bootstrap params
    #     bootstrap_params = np.empty((num_boots, len(model.param_names)))

    #     for iter in range(num_boots):
    #         #for every point on the x axis, sample a y value within the y errorbars
    #         # y is already ordered, do a y + error
    #         # Assume errors are gaussian, sample 1 value from a random distribution of mu y, sigma = yerr
    #         resampled_y = np.random.normal(loc=flux, scale=fluxerr)

    #         if iter < 5 and self.verbose:
    #             plt.clf()
    #             plt.plot(x, resampled_y, label='resampled')
    #             plt.plot(x, flux, label='emission line')
    #             plt.title(f'Resampled #{i}')
    #             plt.legend()
    #             plt.show()

    #         if len(param_kwargs) == 7:  # double Gaussian
    #             Param = self.doubleLineParam.copy()
    #         elif len(param_kwargs) == 4:  # single Gaussian
    #             Param = self.lineParam.copy()
    #         else:
    #             raise ValueError('Please use either the single or double Gauss model.')

    #         boot_result = model.fit(
    #             np.array(resampled_y),
    #             Param,
    #             x=x,
    #             weights=1 / fluxerr,
    #             scale_covar=False,
    #             method=method
    #         )

    #         bootstrap_params[iter] = [boot_result.params[name].value for name in model.param_names]

    #     err_dict = dict(zip(model.param_names, bootstrap_params.std(axis=0)))
    #     return err_dict
def bootstrap_err(self, x, model, flux, fluxerr, params, num_boots, method='leastsq'):
        gaussflux = params['flux']
        paramNames = model.param_names
        param_kwargs = {name: params[name] for name in paramNames}
        residuals = gaussflux - flux
        if self.verbose:
            plt.clf()
            plt.title('residuals')
            plt.plot(np.arange(len(residuals)), residuals, label='residuals')
            plt.plot(np.arange(len(residuals)), gaussflux, label='gaussflux')
            plt.plot(np.arange(len(residuals)), flux, label='flux')
            plt.legend()
            plt.show()

        nblocks = 30
        if len(flux)%nblocks!=0:
            remainder = len(flux) % nblocks
            circle_back = nblocks - remainder
            residuals = np.concatenate([residuals, residuals[:circle_back]])
        split_resid = np.split(residuals, nblocks)
        
        # Preallocate bootstrap params
        bootstrap_params = np.empty((num_boots, len(model.param_names)))

        for i in range(num_boots):
            #resample blocks
            resample_idx = np.random.randint(0, len(split_resid), size=len(split_resid))
            resample = np.array(split_resid)[resample_idx]
            #concatenate blocks and cut to length of gaussflux
            resample = np.ravel(resample)[:len(gaussflux)]
            #add to gaussflux
            resampled_y = gaussflux+resample # Resampled signal
            if i < 5 and self.verbose:
                plt.clf()
                plt.plot(x, resampled_y)
                plt.title(f'Resampled #{i}')
                plt.show()
            if len(param_kwargs) == 7:  # double Gaussian
                Param = self.doubleLineParam.copy()
            elif len(param_kwargs) == 4:  # single Gaussian
                Param = self.lineParam.copy()
            else:
                raise ValueError('Please use either the single or double Gauss model.')

            boot_result = model.fit(
                resampled_y,
                Param,
                x=x,
                weights=1 / fluxerr,
                scale_covar=False,
                method=method
            )
            bootstrap_params[i] = [boot_result.params[name].value for name in model.param_names]

        err_dict = dict(zip(model.param_names, bootstrap_params.std(axis=0)))
        return err_dict

## Functions

In [460]:
def get_asnfilenames(fuvFile, datadir):
    """
    A function to fetch the right association filenames for convolution based on fits header information

    Parameters:
    fuvFile: a FITS HDU file with a SCI header
    datadir: the directory in which the association files are stored

    Returns:
    disptab_file (str): the name of the associated dispersion table file
    lsf_file (str): the name of the associated line spread function file based on 
                    filter, lifetime position, and central wavelength
    """
    fuvHeader0 = fits.getheader(fuvFile, ext=0)

    disptab_file = fuvHeader0['DISPTAB'].split("$")[1]
    grating = fuvHeader0['OPT_ELEM']
    cenwave = fuvHeader0['CENWAVE']
    lpPos = fuvHeader0['LIFE_ADJ']
    lsf_file = os.path.join(datadir,f"aa_LSFTable_{grating}_{cenwave}_LP{lpPos}_cn.dat")
    disptab_file = os.path.join(datadir,disptab_file)
    return lsf_file, disptab_file

def get_disp_params(disptab, cenwave, filt, segment):
    data = disp_df.loc[disptab]
    wh_disp = np.where(
                (data["CENWAVE"] == cenwave)
                & (data["SEGMENT"] == segment)
                & (data["OPT_ELEM"] == filt)
            )[0]# 0 is needed as this returns nested list [[arr]]
    disp_coeff = data['COEFF'][wh_disp][0]# 0 is needed as this returns nested list [[arr]]
    d_tv03 = data["D_TV03"][wh_disp]  # Offset from WCA to PSA in Thermal Vac. 2003 data
    d_orbit = data["D"][wh_disp]  # Current offset from WCA to PSA
    delta_d = d_tv03 - d_orbit
    disp_coeff = disp_coeff-delta_d
    return disp_coeff
    
def redefine_lsf(lsf_file, disptab, cenwave, filter, segment):
    """
    Helper function to convolve_lsf(). Converts the LSF kernels in the LSF file from a fn(pixel) -> fn(λ)\
    which can then be used by convolve_lsf() and re-bins the kernels.
    Parameters:
    lsf_file (str): path to your LSF file
    cenwave (str): Cenwave for calculation of dispersion relationship
    disptab (str): path to your DISPTAB file
    
    Returns:
    new_lsf (numpy.ndarray): Remapped LSF kernels.
    new_w (numpy.ndarray): New LSF kernel's LSF wavelengths.
    step (float): first order coefficient of the FUVA dispersion relationship; proxy for Δλ/Δpixel.
    """

    # Read in the dispersion relationship here for the segments
    if segment == 'FUVB':
        if (cenwave != 1105) & (cenwave != 800):
            disp_coeff = get_disp_params(disptab, cenwave, filter, segment)
    else:
        disp_coeff = get_disp_params(disptab, cenwave, filter, segment)

    # Get the step size info from the FUVA 1st order dispersion coefficient
    step = disp_coeff[1]

    # Read in the lsf file
    data = lsf_df.loc[lsf_file]
    lsf=data['lsf']
    pix=data['pix']
    w=data['w']

    # take median spacing between original LSF kernels
    deltaw = np.median(np.diff(w))

    if (deltaw < len(pix) * step * 2):  # resamples if the spacing of the original LSF wvlns is too narrow
        # this is all a set up of the bins we want to use
        # The wvln difference between kernels of the new LSF should be about twice their width
        new_deltaw = int(len(pix) * step * 2.0)  
        new_nw = (int(round((max(w) - min(w)) / new_deltaw)) + 1)  # nw = number of LSF wavelengths
        
        new_w = min(w) + np.arange(new_nw) * new_deltaw  # new version of lsf_wvlns
        
        # populating the lsf with the proper bins
        new_lsf = np.zeros((len(pix), new_nw))  # empty 2-D array to populate

        for i, current_w in enumerate(new_w):
            dist = abs(current_w - w)  # Find closest original LSF wavelength to new LSF wavelength
            lsf_index = np.argmin(dist)
            orig_lsf_wvln_key = lsf.keys()[lsf_index]  # column name corresponding to closest orig LSF wvln
            new_lsf[:, i] = np.array(lsf[orig_lsf_wvln_key])  # assign new LSF wvln the kernel of the closest original lsf wvln
    else:
        new_lsf = lsf
        new_w = w
    return new_lsf, new_w, step

def convolve_lsf(wavelength, spec, lsf_file, disptab, cenwave, filter, segment, verbose=False):

    """
    Main function; Convolves an input spectrum - i.e. template or STIS spectrum - with the COS LSF.
    Parameters:
    wavelength (list or array): Wavelengths of the spectrum to convolve.
    spec (list or array): Fluxes or intensities of the spectrum to convolve.
    specerr (list or array): Flux errors of the spectrum to convolve.
    cenwave (str): Cenwave for calculation of dispersion relationship
    lsf_file (str): Path to your LSF file
    disptab (str): Path to your DISPTAB file
    Returns:
    wave_cos (numpy.ndarray): Wavelengths of convolved spectrum.!Different length from input wvln
    final_spec (numpy.ndarray): New LSF kernel's LSF wavelengths.!Different length from input spec
    """
    # First calls redefine to get all LSF kernels transformed to the right wavelength scales
    new_lsf, new_w, step = redefine_lsf(lsf_file, disptab, cenwave, filter, segment)

    # sets up new wavelength scale used in the convolution
    nstep = round((max(wavelength) - min(wavelength)) / step) - 1
    wave_cos = min(wavelength) + np.arange(nstep) * step #construct the goal LSF wavelength to transform the LSF flux to

    # interpolate input spectrum onto the LSF's wavelength resolution
    interp_func = interp1d(wavelength, spec)  # builds up interpolated function from input spectrum
    spec_cos = interp_func(wave_cos)  # lower res version of the same input spectrum
    final_spec = interp_func(wave_cos)  # initialize convolution spectrum

    # interp_errf = interp1d(wavelength, specerr)
    # specerr_cos = interp_errf(wave_cos)
    # final_specerr = interp_errf(wave_cos)

    peak_wavelength = wavelength[np.argmax(spec)]
    #===============THIS=====================
    #best_kernel_idx = np.argmin(np.abs(new_w-peak_wavelength))
    #current_lsf = new_lsf[:, best_kernel_idx]

    #===========OR THAT========================
    #interpolate nearest LSF kernels to emission line wavelength space
    if peak_wavelength <= new_w[0]:
        current_lsf = new_lsf[:, 0]
    elif peak_wavelength >= new_w[-1]:
        current_lsf = new_lsf[:, -1]
    else:
        lower_idx = np.searchsorted(new_w, peak_wavelength) - 1
        upper_idx = lower_idx + 1

        wl_low = new_w[lower_idx]
        wl_high = new_w[upper_idx]
        lsf_low = new_lsf[:, lower_idx]
        lsf_high = new_lsf[:, upper_idx]

        # Linear interpolation weight
        w = (peak_wavelength - wl_low) / (wl_high - wl_low)
        interp_lsf = (1 - w) * lsf_low + w * lsf_high
        current_lsf = interp_lsf/np.sum(interp_lsf)

    #=======PLOTTING FOR DIAGNOSTICS=================

    # low_lambda_axis  = wl_low  + np.arange(-lsf_low.size  // 2, lsf_low.size  // 2) * step
    # high_lambda_axis = wl_high + np.arange(-lsf_high.size // 2, lsf_high.size // 2) * step
    # lambda_axis      = peak_wavelength + np.arange(-current_lsf.size // 2, current_lsf.size // 2) * step

    # low_lsf_shifted  = (lsf_low  / lsf_low.max())  * spec_cos.max()
    # high_lsf_shifted = (lsf_high / lsf_high.max()) * spec_cos.max()
    # lsf_shifted      = (current_lsf      / current_lsf.max())      * spec_cos.max()

    # plt.clf()
    # plt.plot(wave_cos, spec_cos, label='Interpolated Spectrum (spec_cos)')
    # plt.plot(low_lambda_axis, low_lsf_shifted, '--', label=f'LSF @ {wl_low:.1f} Å')
    # plt.plot(high_lambda_axis, high_lsf_shifted, '--', label=f'LSF @ {wl_high:.1f} Å')
    # plt.plot(lambda_axis, lsf_shifted, '--', label='Interpolated LSF')
    # plt.legend()
    # plt.show()

    final_spec = convolve(
                    spec_cos,
                    current_lsf,  # Applies the actual convolution
                    boundary="extend",
                    normalize_kernel=True,
                )
    #one final interpolation to make arrays the same length as the input arrays
    xnew = np.linspace(min(wave_cos), max(wave_cos), len(wavelength))
    final_interpfunc = interp1d(wave_cos, final_spec)
    
    conv_spec = final_interpfunc(xnew)
    return xnew, conv_spec

def single_bootstrap(args):
    warnings.filterwarnings(
        "ignore",
        message="Using UFloat objects with std_dev==0 may give unexpected results.",
        category=UserWarning,
    )
    warnings.simplefilter('ignore', AstropyDeprecationWarning)

    x, gaussflux, model, split_resid, fluxerr, param_kwargs, method, verbose, conv_args= args
        
    resample_idx = np.random.randint(0, len(split_resid), size=len(split_resid))
    resample = np.array(split_resid)[resample_idx]
    #concatenate blocks and cut to length of gaussflux
    resample = np.ravel(resample)[:len(gaussflux)]
    #add to gaussflux
    resampled_y = gaussflux+np.sqrt((resample**2+fluxerr**2)) # Resampled signal
    peak = np.max(resampled_y)
    Param = model.make_params(**param_kwargs)
    # Pick correct parameter set
    eps = 1e-12
    if len(param_kwargs) == 7:
        Param['amp1'].set(min=0.3*peak, max=max(2*peak, eps))
        Param['amp2'].set(min=0.2*peak, max=max(peak, 0.8*eps)) #just need amp2 to be smaller
        Param['sig1'].set(min=0.2*Param['sig1'].value, max=2*Param['sig1'].value)
        Param['sig2'].set(min=0.3*Param['sig2'].value, max=3*Param['sig2'].value)
        Param['cen2'].set(min=Param['cen1'].value-1, max=Param['cen1'].value+1)
        Param['cen1'].set(min=Param['cen2'].value-1, max=Param['cen2'].value+1)
        Param['bgl'].set(min=0.6*Param['bgl'].value, max=1e-12)
    elif len(param_kwargs) == 4:
        Param['amp'].set(min=0, max=max(peak*1.2, eps))
        Param['sig'].set(min=0.0, max=3) #1 wavelength ~ 200km/s radial velocity
        Param['cen'].set(min=np.min(x), max=Param['cen'].value+3)
        Param['bgl'].set(min=Param['bgl'].value*0.8, max=1e-10)
    else:
        raise ValueError('Please use either the single or double Gauss model.')

    # Fit model
    lsf_file, disptab, cenwave, filt, segment = conv_args
    boot_result = model.fit(
        resampled_y,
        Param,
        x=x,
        weights=1/resample,
        scale_covar=False,
        method=method,
        lsf_file=lsf_file, 
        disptab=disptab, 
        cenwave=cenwave, 
        filt=filt, 
        segment=segment
    )
    if verbose:
        print(boot_result.covar)
    return [boot_result.params[name].value for name in model.param_names]


In [461]:
class spectrum():
    def __init__(self, files, linelist_file, dataDir, convolve=False):
    
        """
        Initialize spectrum, unpack spectra into useable arrays
        
        Parameters:
        - files: a list of .fits files from HST, each with a header hdu, and a spectrum BinTable hdu
        """
        self.flux_units = (u.erg/(u.AA*u.s*(u.cm**2)))
        self.wavelength_units = u.AA
        self.linelist = None
        self.linelist = pd.read_csv(linelist_file, skiprows=2)
        #construct dataframe with filename, relevant header information, and spectrum information for each file.
        #Then add to a list of dataframes from files
        rows = []
    
        for file in files:
            #Use np.ravel here because the wavelength in the table  is a list in one table cell (e.g. [[1, 2, 3]])
            INST_head = fits.getheader(file, ext=0)
            SCI_head = fits.getheader(file, ext=1)
            spectrum_x1d = Table.read(file, hdu=1)

            LSF_file, disptab_file = get_asnfilenames(fuvFile=file, datadir=dataDir)
            cenwave = INST_head['CENWAVE']

            if "cspec" in file:
                date = SCI_head['DATE-BEG'][:10]
                segments = None
            elif "x1dsum" in file:
                date = SCI_head['DATE-OBS'] 
                instrument = INST_head['INSTRUME']
                if len(spectrum_x1d['WAVELENGTH']):
                    segmenta = [spectrum_x1d['WAVELENGTH'][spectrum_x1d['SEGMENT']=='FUVA'].min(), spectrum_x1d['WAVELENGTH'][spectrum_x1d['SEGMENT']=='FUVA'].max()]
                    segmentb = [spectrum_x1d['WAVELENGTH'][spectrum_x1d['SEGMENT']=='FUVB'].min(), spectrum_x1d['WAVELENGTH'][spectrum_x1d['SEGMENT']=='FUVB'].max()]
                    segments = [segmenta, segmentb]
                else:
                    segments = None
            try:
                instrument_filter = INST_head['OPT_ELEM']
            except KeyError:
                instrument_filter = INST_head['GRATING'] #in cspec header

            x1d_wavelength = np.ravel(spectrum_x1d['WAVELENGTH'])/self.wavelength_units
            x1d_flux = np.ravel(spectrum_x1d['FLUX'])/self.flux_units
            x1d_fluxerr = np.ravel(spectrum_x1d['ERROR'])/self.flux_units
            self.fileInfo_keys = ['filename', 'obsdate', 'instrument', 'filter', 'dispfile', 'LSFfile', 'cenwave']
            self.specInfo_keys = ['wavelength', 'flux', 'fluxerr', 'velocity', 'segment']
            self.lineInfo_keys = ['H2line_idx', "labline", "transition_prob"]
            df_data = {'filename':file, 
                    'obsdate':date,
                    'instrument':instrument, 
                    'filter':instrument_filter,
                    'dispfile':disptab_file,
                    'LSFfile': LSF_file,
                    'cenwave':cenwave,
                    'wavelength':x1d_wavelength*self.wavelength_units, 
                    'flux': x1d_flux*self.flux_units, 
                    'fluxerr':x1d_fluxerr*self.flux_units,
                    'segment': segments}
            rows.append(pd.Series(df_data))
            
        self.spec_df = pd.DataFrame(rows)
        drop_idx = []
        for i in range(len(self.spec_df)-1):
            if len(self.spec_df.iloc[i]['wavelength'])<1:
                drop_idx.append(i)
        self.spec_df.drop(labels=drop_idx, inplace=True)
        self.spec_df.reset_index(drop=True, inplace=True)
        self.spec_df['obsdate'] = self.spec_df['obsdate'].astype(str)
    
    def mask_df(self, df, mask_dict):
        columns = mask_dict.keys()
        values = mask_dict.values()
        mask = np.logical_and.reduce([(df[col] == val) for col, val in zip(columns, values)])
        filtered = df[mask]
        return filtered

    def coaddwave(self, df, columns=['obsdate']):
        """
        Co-add spectra using the spectrum1D function.

        Parameters:
        - df: The dataframe to use for co-adding. Must contain 'wavelength', 
              'flux', and 'fluxerr' columns at least. If using velocity_space, must contain 'rest_wavelength' column.
        - columns: the columns of unique combinations to apply coadding to.

        Returns:
        - coadded_spectrum: spectrum dataframe
        """

        drop_indices = []
        combinations = df[columns].drop_duplicates()
        for _, row in combinations.iterrows():
            unique_values = [row[col] for col in columns]

            # Filter the group
            mask = np.logical_and.reduce([(df[col] == val) for col, val in zip(columns, unique_values)])
            group = df[mask]

            resampler = LinearInterpolatedResampler(extrapolation_treatment='zero_fill')
            wavelengths = group['wavelength'].values
            fluxes = group['flux'].values
            fluxerrs = group['fluxerr'].values
            
            template_axis = wavelengths[0][np.argsort(wavelengths[0])]
            template_flux = fluxes[0][np.argsort(wavelengths[0])]
            template_fluxerr = fluxerrs[0][np.argsort(wavelengths[0])]
            
            coadded = Spectrum1D(
                flux=template_flux,
                spectral_axis=template_axis,
                uncertainty=StdDevUncertainty(template_fluxerr)
            )

            for i in np.arange(0, len(wavelengths)):
                x = wavelengths[i]
                flux = fluxes[i]
                err = fluxerrs[i]
                sort_idx = np.argsort(x)
                spec = Spectrum1D(spectral_axis=x[sort_idx], 
                                    flux=flux[sort_idx], 
                                    uncertainty=StdDevUncertainty(err[sort_idx])
                                )
                
                spec_resampled = resampler(spec, coadded.spectral_axis)
                coadded += spec_resampled
            
            first_index = group.index[0]
            df.at[first_index, 'wavelength'] = coadded.wavelength
            df.at[first_index, 'flux'] = coadded.flux
            df.at[first_index, 'fluxerr'] = coadded.uncertainty.quantity.to(self.flux_units)
            drop_indices.extend(group.index[1:])
        
        df.drop(drop_indices, inplace=True)
        df.reset_index(inplace=True, drop=True)
        #return stacked_df

    def H2lines_fromspectrum(self, df):
        """
        Get the H2 line peaks from the spectrum, and add the peaks to the spectrum dataframe

        Parameters:
        - linelist: an array of

        Returns:
        - Line peak indices and the associated transitions added to the original dataframe
        """
        df['H2line_idx'] = pd.Series([None] * len(df), dtype=object)
        df["[nu', J']"] = pd.Series([None] * len(df), dtype=object)
        df["labline"] = pd.Series([None] * len(df), dtype=object)
        df["transition_prob"] = pd.Series([None] * len(df), dtype=object)
        for i, row in df.iterrows():            
            #Find the emission lines associated with H2 fluorescence
            emission_table_idx = [] #wavelength array index
            nu_J = [] #transition of line from linelist
            lab_lines = [] #lab wavelength of emission line from linelist
            transition_prob = []

            for il, l in enumerate(self.linelist['lab_lambda'].values):
                wavelength = row['wavelength']/u.AA
                diffs = np.abs(wavelength-l)
                mask = diffs <= 0.2
                if np.any(mask):
                    min_idx = np.argmin(diffs)
                    emission_table_idx.append(min_idx)
                    nu_J.append(self.linelist.loc[il, '[nu\', J\']'])
                    transition_prob.append(self.linelist.loc[il, 'transition_prob'])
                    lab_lines.append(l)
            df.at[i, 'H2line_idx'] = np.array(emission_table_idx)
            df.at[i, "[nu', J']"] = nu_J
            df.at[i, 'labline'] = lab_lines
            df.at[i, 'transition_prob'] = transition_prob
        return df

    def radial_velocity(self, wavelengths, labline):
        velocity = (wavelengths - labline*u.AA)/(labline*u.AA) * const.c.to(u.km/u.s)
        return velocity
    
    def get_lines(self, df):
        line_width = 150 #150 points on either side of the central line
        transitions = set(self.linelist["[nu\', J\']"])
        line_keys = self.fileInfo_keys+self.specInfo_keys+self.lineInfo_keys
        line_dict = {key: [] for key in line_keys}
        line_dict['transition'] = []
        for target_nuJ in transitions:
            for _, row in df.iterrows():
                if target_nuJ in row["[nu', J']"]:
                    nuJ_array = np.atleast_1d(row["[nu', J']"])
                    h2_idx_array = np.atleast_1d(row['H2line_idx'])
                    targ_idx = np.flatnonzero(nuJ_array == target_nuJ)
                    line_idx = h2_idx_array[targ_idx] #the index of the H2 line of interest in every wavelength coadded spectrum

                    for i, l in enumerate(line_idx):
                        start = max(0, l - line_width)
                        end = min(len(row['wavelength']), l + line_width)  
                        #Spectrum information
                        waves = row['wavelength'][start:end]
                        line_dict['wavelength'].append(waves)
                        line_dict['flux'].append(row['flux'][start:end])
                        line_dict['fluxerr'].append(row['fluxerr'][start:end])
                        velocity = self.radial_velocity(waves, row['labline'][targ_idx[i]])
                        line_dict['velocity'].append(velocity)
                        if row['segment']:
                            if row['segment'][0][0] <= row['labline'][targ_idx[i]] <= row['segment'][0][1]:
                                line_dict['segment'].append('FUVA')
                            elif row['segment'][1][0] <= row['labline'][targ_idx[i]] <= row['segment'][1][1]:
                                line_dict['segment'].append('FUVB')
                            else:
                                line_dict['segment'].append(None)
                        # line_dict['velocity'].append(np.nan)
                        #File information
                        for key in self.fileInfo_keys:
                            line_dict[key].append(row[key])
                        #Line information
                        line_dict['transition'].append(target_nuJ)
                        for key in self.lineInfo_keys:    
                            line_dict[key].append(np.array(row[key])[targ_idx[i]])
        line_df = pd.DataFrame(line_dict)
        return line_df

In [468]:
class ModelFitting():
    def __init__(self, df, mask_dict, verbose=False):
        #operate largely without units
        df['BIC1'] = pd.Series([None] * len(df), dtype=object)
        df['params1'] = pd.Series([None] * len(df), dtype=object)
        df['BIC2'] = pd.Series([None] * len(df), dtype=object)
        df['params2'] = pd.Series([None] * len(df), dtype=object)
        self.df = df
        self.mask_dict = mask_dict
        self.lineModel = Model(self.gaussian_conv, independent_vars=['x', 'lsf_file', 'disptab', 'cenwave', 'filt', 'segment'])
        self.doubleLineModel = Model(self.double_gauss_conv, independent_vars=['x', 'lsf_file', 'disptab', 'cenwave', 'filt', 'segment'])
        self.flux_units = (u.erg/(u.AA*u.s*(u.cm**2)))
        self.wavelength_units = u.AA
        self.verbose = verbose

    def mask_contamination(self, labline, mask_dict):
        mask_idx = mask_dict[labline]
        if isinstance(mask_idx, list):
            mask=np.ones(300, dtype=bool)
            mask[mask_idx[0]:mask_idx[1]] = False
            return mask
        else:
            mask=np.zeros(300, dtype=bool)
            mask[mask_idx:] = True
            return mask
        
    def gaussian(self, x, amp, sig, cen, bgl):
        x = np.array(x)
        y = amp * np.exp(-((x - cen)**2.0)/(2.0 * (sig**2.0))) + bgl
        return y

    def gaussian_conv(self, x, amp, sig, cen, bgl, lsf_file, disptab, cenwave, filt, segment):
        x = np.array(x)
        y = amp * np.exp(-((x - cen)**2.0)/(2.0 * (sig**2.0))) + bgl
        
        _, y_conv = convolve_lsf(x, y, lsf_file, disptab, cenwave, filt, segment)
        
        return y_conv

    def double_gauss_conv(self, x, amp1, sig1, cen1, amp2, sig2, cen2, bgl, lsf_file, disptab, cenwave, filt, segment):
        y = self.gaussian(x, amp1, sig1, cen1, 0) + self.gaussian(x, amp2, sig2, cen2, 0) + bgl
        _, y_conv = convolve_lsf(x, y, lsf_file, disptab, cenwave, filt, segment, self.verbose)
        return y_conv
            
    def gauss_err(self, x, y, yerr, params, paramerrs, cov, conv_args):
        """
        Calculate propagated error for a Gaussian model with background.
        Parameters assumed: [amp, sig, cen, bgl]
        """
        lsf_file, disptab, cenwave, filt, segment = conv_args

        amp, sig, cen, _ = [params[k] for k in self.lineModel.param_names]
        err_vals = [paramerrs[k] for k in self.lineModel.param_names]
        amp_err, sig_err, cen_err, bgl_err = err_vals
        
        # Compute the Gaussian (without background)
        gauss = amp * np.exp(-(x - cen)**2 / (2*sig**2))
        
        # Compute partial derivatives
        dfdA = np.exp(-(x - cen)**2 / (2*sig**2))
        dfdmu = gauss * np.abs(x - cen) / (sig**2)
        dfdsig = gauss * ((x - cen)**2 / (sig**3))
        dfdbgl = np.ones_like(x)

        if self.verbose:
            dfdA_term = (dfdA * amp_err)**2
            dfdmu_term = (dfdmu * cen_err)**2
            dfdsig_term = (dfdsig * sig_err)**2
            dfdbgl_term = (dfdbgl * bgl_err)**2

            plt.plot(x, np.sqrt(dfdA_term), label="amp contribution")
            plt.plot(x, np.sqrt(dfdmu_term), label="cen contribution")
            plt.plot(x, np.sqrt(dfdsig_term), label="sigma contribution")
            plt.plot(x, np.sqrt(dfdbgl_term), label="background contribution")
            plt.legend()
            plt.show()

        # Propagate uncertainties (assuming parameters uncorrelated)
        if cov is not None:
            J = np.column_stack([dfdA, dfdsig, dfdmu, dfdbgl])
            cov_matrix = np.asarray(cov)
            spec_var = np.einsum('ij,jk,ik->i', J, cov_matrix, J)
            specerr = np.sqrt(spec_var)

        elif all(v is not None for v in err_vals):
            # Assume uncorrelated parameters
            specerr = np.sqrt((dfdA * amp_err)**2 +
                            (dfdmu * cen_err)**2 +
                            (dfdsig * sig_err)**2 +
                            (dfdbgl * bgl_err)**2)
        else:
            print('Bootstrapping errors')
            _, specerr = self.bootstrap_err(x, self.lineModel, y, yerr, params, conv_args)
        
        xconv, conv_err = convolve_lsf(x, specerr, lsf_file, disptab, cenwave, filt, segment)

        return xconv, conv_err

    def double_gauss_err(self, x, y, yerr, params, paramerrs, cov, conv_args):
        """
        Calculate propagated error for a double Gaussian with background.
        Parameters: [amp1, sig1, cen1, amp2, sig2, cen2, bgl]
        """
        lsf_file, disptab, cenwave, filt, segment = conv_args

        # Extract parameter values and errors
        pnames = self.doubleLineModel.param_names
        vals = [params[k] for k in pnames]
        err_vals = [paramerrs[k] for k in pnames]
        amp1_err, sig1_err, cen1_err, amp2_err, sig2_err, cen2_err, bgl_err = err_vals
        amp1, sig1, cen1, amp2, sig2, cen2, _ = vals

        # Compute the Gaussians without background
        g1 = amp1 * np.exp(-(x - cen1)**2 / (2*sig1**2))
        g2 = amp2 * np.exp(-(x - cen2)**2 / (2*sig2**2))
        
        # Partial derivatives for Gaussian 1
        dfdA1 = np.exp(-(x - cen1)**2 / (2*sig1**2))
        dfdmu1 = g1 * (x - cen1) / (sig1**2)
        dfdsig1 = g1 * ((x - cen1)**2 / (sig1**3))
        
        # Partial derivatives for Gaussian 2
        dfdA2 = np.exp(-(x - cen2)**2 / (2*sig2**2))
        dfdmu2 = g2 * (x - cen2) / (sig2**2)
        dfdsig2 = g2 * ((x - cen2)**2 / (sig2**3))
        
        # Background derivative
        dfdbgl = np.ones_like(x)
        
        # Combine propagated errors
        if cov is not None:
            J = np.column_stack([dfdA1, dfdsig1, dfdmu1, dfdA2, dfdsig2, dfdmu2, dfdbgl])
            
            cov_matrix = np.asarray(cov)
            if cov_matrix.shape != (7, 7):
                raise ValueError(f"Covariance matrix shape {cov_matrix.shape} is not (4,4)")
            
            spec_var = np.einsum('ij,jk,ik->i', J, cov, J)
            specerr = np.sqrt(spec_var)
        elif all(v is not None for v in err_vals):
            #Assuming parameters uncorrelated
            specerr = np.sqrt(
                (dfdA1 * amp1_err)**2 +
                (dfdmu1 * cen1_err)**2 +
                (dfdsig1 * sig1_err)**2 +
                (dfdA2 * amp2_err)**2 +
                (dfdmu2 * cen2_err)**2 +
                (dfdsig2 * sig2_err)**2 +
                (dfdbgl * bgl_err)**2
            )
        else:
            print('Bootstrapping errors')
            _, specerr = self.bootstrap_err(x, self.doubleLineModel, y, yerr, params, conv_args)

        xconv, conv_err = convolve_lsf(x, specerr, lsf_file, disptab, cenwave, filt, segment)
        
        return xconv, conv_err

    def fit_oneGauss(self, x, y, yerr, conv_args):
        #mask the spurious lines in each unique progression, labline combo
        peak_idx = np.argmax(y)
        peak = y[peak_idx]
        cen = x[peak_idx]
        bgl = np.median(y[0:10])
        #single gaussian fit
        self.lineParam = self.lineModel.make_params(amp=peak,
                                        sig=1,
                                        cen=cen,
                                        bgl=bgl
                                        )
        eps=1e-12
        self.lineParam['amp'].set(min=0, max=max(peak*1.2, eps))
        self.lineParam['sig'].set(min=0.0, max=3) #1 wavelength ~ 200km/s radial velocity
        self.lineParam['cen'].set(min=np.min(x), max=self.lineParam['cen']+3)
        self.lineParam['bgl'].set(min=bgl*0.8, max=1e-10)
        lsf_file, disptab, cenwave, filt, segment = conv_args

        lineFit = self.lineModel.fit(y, 
                                self.lineParam, 
                                x=x, 
                                weights=1/yerr,
                                scale_covar=False,
                                nan_policy='omit',
                                lsf_file=lsf_file, 
                                disptab=disptab, 
                                cenwave=cenwave, 
                                filt=filt, 
                                segment=segment
                                )
        bestParam = lineFit.params
        param_dict = {param:bestParam[param].value for param in self.lineModel.param_names}
        err_dict = {param:bestParam[param].stderr for param in self.lineModel.param_names}
        cov = lineFit.covar
        xconv, fiterr = self.gauss_err(x, y, yerr, param_dict, err_dict, cov, conv_args)
        if self.verbose:
            print(f'Single gauss fit {lineFit.success}')
            print(lineFit.message)
            fit = self.gaussian_conv(x,
                                    bestParam['amp'].value, 
                                    bestParam['sig'].value, 
                                    bestParam['cen'].value, 
                                    bestParam['bgl'].value,
                                    lsf_file=lsf_file, 
                                    disptab=str(disptab), 
                                    cenwave=cenwave, 
                                    filt=filt, 
                                    segment=segment)
            plt.clf()
            plt.plot(x,y)
            plt.plot(x, fit)
            plt.fill_between(x, fit-fiterr, fit+fiterr, alpha=0.2)
            plt.show()
        bic = lineFit.bic
        return param_dict, err_dict, bic, fiterr

    def fit_twoGauss(self, x, y, yerr, bestPar, conv_args):
        peak_idx = np.argmax(y)
        peak = y[peak_idx]
        #double gaussian fit. Uses the single gauss as the initial guess to test whether a second peak is necessary.
        #additional test on top of the bic
        #first is left, second is right gauss
        self.doubleLineParam = self.doubleLineModel.make_params(amp1=peak*1.2,
                                                    sig1=bestPar['sig']*0.8, 
                                                    cen2=x[peak_idx], #source of emission is sum over the whole disk
                                                    amp2=peak*0.3, 
                                                    sig2=1, #200 km/s rotating disk
                                                    cen1=bestPar['cen']-5e-2, #Blue-shifted component of emitted light
                                                    bgl=bestPar['bgl']
                                                    )
        eps = 1e-12
        #first is tall narrow one, second gauss is the small broad one
        self.doubleLineParam['amp1'].set(min=0.3*peak, max=max(2*peak, eps))
        self.doubleLineParam['amp2'].set(min=0.2*peak, max=max(peak, 0.8*eps)) #just need amp2 to be smaller
        self.doubleLineParam['sig1'].set(min=0.2*bestPar['sig'], max=2*bestPar['sig'])
        self.doubleLineParam['sig2'].set(min=0.3*bestPar['sig'], max=3*bestPar['sig'])
        self.doubleLineParam['cen2'].set(min=bestPar['cen'], max=bestPar['cen']+1)
        self.doubleLineParam['cen1'].set(min=bestPar['cen']-1, max=bestPar['cen'])
        self.doubleLineParam['bgl'].set(min=0.6*bestPar['bgl'], max=1e-12)
        
        lsf_file, disptab, cenwave, filt, segment = conv_args
        doubleFit = self.doubleLineModel.fit(y,
                                    self.doubleLineParam,
                                    x=x,
                                    weights=1/yerr,
                                    scale_covar=False,
                                    lsf_file=lsf_file, 
                                    disptab=disptab, 
                                    cenwave=cenwave, 
                                    filt=filt, 
                                    segment=segment
                                    )
        
        best2par = doubleFit.params
        param_dict = {param:best2par[param].value for param in self.doubleLineModel.param_names}
        err_dict = {param:best2par[param].stderr for param in self.doubleLineModel.param_names}
        cov = doubleFit.covar
        xconv, fiterr = self.double_gauss_err(x, y, yerr, param_dict, err_dict, cov, conv_args)
        if self.verbose:
            print(f'Double gauss fit {doubleFit.success}')
            print(doubleFit.message)
            fit = self.double_gauss_conv(x,
                                    best2par['amp1'].value, 
                                    best2par['sig1'].value, 
                                    best2par['cen1'].value, 
                                    best2par['amp2'].value, 
                                    best2par['sig2'].value, 
                                    best2par['cen2'].value, 
                                    best2par['bgl'].value,
                                    lsf_file=lsf_file, 
                                    disptab=str(disptab), 
                                    cenwave=cenwave, 
                                    filt=filt, 
                                    segment=segment)
            gauss1 = self.gaussian(x, best2par['amp1'].value, best2par['sig1'].value, best2par['cen1'].value, best2par['bgl'].value)
            gauss2 = self.gaussian(x, best2par['amp2'].value, best2par['sig2'].value, best2par['cen2'].value, best2par['bgl'].value)

            plt.clf()
            plt.plot(x,y, label='data')
            plt.plot(x, fit, label='fit')
            plt.fill_between(x, fit-fiterr, fit+fiterr, alpha=0.2)
            plt.plot(x, gauss1, label='component 1')
            plt.plot(x, gauss2, label='component 2')
            plt.show()
        
        bic = doubleFit.bic
        return param_dict, err_dict, bic, fiterr, xconv
    
    def mc_err_with_cov(self, x, params, model, cov, conv_args):
        """
        Monte Carlo error propagation for a Gaussian model using correlated parameter sampling.

        Parameters
        ----------
        x : array
            Wavelength or x-axis values.
        params : dict or lmfit Parameters
            Best-fit parameter values.
        cov : 2D array
            Parameter covariance matrix in the same order as param_names.
        Returns
        -------
        mean_gauss : array
            Mean Gaussian profile over samples.
        gauss_err : array
            1 std uncertainty at each x point.
        """
        lsf_file, disptab, cenwave, filt, segment = conv_args
        # Mean parameter vector
        param_names = model.param_names
        mean_vec = np.array([params[name] for name in param_names], dtype=float)

        # Draw correlated samples from multivariate Gaussian
        samples = np.random.multivariate_normal(mean_vec, cov, size=1000)

        # Evaluate Gaussian for each sampled parameter set
        if len(param_names) == 4:  # Single Gaussian
            gaussians = np.array([
                                self.gaussian_conv(x, *s, lsf_file, disptab, cenwave, filt, segment)
                                for s in samples
                            ])

        elif len(param_names) == 7:  # Double Gaussian
            gaussians = np.array([
                                self.double_gauss_conv(x, *s, lsf_file, disptab, cenwave, filt, segment)
                                for s in samples
                            ])

        # Compute statistics
        gauss_err = gaussians.std(axis=0)

        return gauss_err

    def bootstrap_err(self, x, model, flux, fluxerr, params, conv_args, num_boots=500, method='leastsq'):
        lsf_file, disptab, cenwave, filt, segment = conv_args
        gaussflux = model.func(x=x, **params, lsf_file=lsf_file, disptab=disptab, cenwave=cenwave, filt=filt, segment=segment)
        paramNames = model.param_names
        param_kwargs = {name: params[name] for name in paramNames}
        residuals = gaussflux - flux
        if self.verbose:
            plt.clf()
            plt.title('residuals for bootstrapping')
            
            plt.plot(np.arange(len(residuals)), gaussflux, label='gaussflux')
            plt.plot(np.arange(len(residuals)), flux, label='flux')
            plt.plot(np.arange(len(residuals)), residuals, label='residuals')
            
            plt.legend()
            plt.show()

        nblocks = 30
        if len(gaussflux)%nblocks!=0:
            remainder = len(gaussflux) % nblocks
            circle_back = nblocks - remainder
            residuals = np.concatenate([residuals, residuals[:circle_back]])
        split_resid = np.split(residuals, nblocks)
        
        args_list = [
            (x, gaussflux, model, split_resid, fluxerr, param_kwargs, method, self.verbose, conv_args)
            for _ in range(num_boots)
        ]

        with mp.Pool(processes=None) as pool:
            results = pool.map(single_bootstrap, args_list)
        
        bootstrap_params = np.array(results)
        err_dict = dict(zip(model.param_names, bootstrap_params.std(axis=0)))
        cov_matrix = np.cov(bootstrap_params.T, bias=False)
        mc_fiterr = self.mc_err_with_cov(x, params, model, cov_matrix, conv_args)
        if self.verbose:
            stds = np.sqrt(np.diag(cov_matrix))
            corr_matrix = cov_matrix / np.outer(stds, stds)
            print('Bootstrap corr = ', corr_matrix)
            fit = gaussflux
            
            plt.clf()
            plt.title('bootstrap errors')
            plt.plot(x, gaussflux, label='best fit gauss')
            plt.fill_between(x, fit-mc_fiterr, fit+mc_fiterr, alpha=0.2, label='mc error')
            plt.plot(x, flux, '--', label='data')
            plt.fill_between(x, flux-fluxerr, flux+fluxerr, alpha=0.2, label='data error')
            plt.xlabel('Wavelength (AA)')
            plt.ylabel(f'Flux ({self.flux_units})')
            plt.legend()
            plt.show()

        return err_dict, mc_fiterr
    
    def fit_lines(self, bootstrap=False):
        self.df['BIC1'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['BIC2'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['params1'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['params2'] = pd.Series([None] * len(self.df), dtype=object)
                
        self.df['conv_wavelength'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['gauss_flux'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['gauss_fluxerr'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['double_gauss_flux'] = pd.Series([None] * len(self.df), dtype=object)
        self.df['double_gauss_fluxerr'] = pd.Series([None] * len(self.df), dtype=object)
        
        for i, row in self.df.iterrows():
            if self.verbose:
                print(f'Fitting line #{i}')
            
            flux = row['flux'].to_value(self.flux_units)
            wavelength = row['wavelength'].to_value(self.wavelength_units)
            fluxerr = row['fluxerr'].to_value(self.flux_units)
            labline = row['labline']

            #mask lines
            x = wavelength.copy()
            y = flux.copy()
            yerr = fluxerr.copy()
            if labline in self.mask_dict.keys():
                mask = self.mask_contamination(labline, self.mask_dict)
                x = x[~mask]
                y = y[~mask]
                yerr = yerr[~mask]

            #set convolution params
            conv_args = [row['LSFfile'], row['dispfile'], row['cenwave'], row['filter'], row['segment']]

            # Fit gaussians
            params1, err_dict1, bic1, yerrGauss = self.fit_oneGauss(x, y, yerr, conv_args)
            params2, err_dict2, bic2, y2errGauss, xconv = self.fit_twoGauss(x, y, yerr, params1, conv_args)
            
            #Construct gaussian
            ygauss = self.gaussian_conv(x=x, lsf_file=row['LSFfile'], disptab=row['dispfile'], cenwave=row['cenwave'], filt=row['filter'], segment=row['segment'], **params1)
            y2gauss = self.double_gauss_conv(x=x, lsf_file=row['LSFfile'], disptab=row['dispfile'], cenwave=row['cenwave'], filt=row['filter'], segment=row['segment'], **params2)

            # Calculate errors
            if bootstrap:
                if self.verbose:
                    print('Calculating bootstrap errors')
                err_dict1, yerrGauss = self.bootstrap_err(x, self.lineModel, y, yerr, params1, conv_args, num_boots=500, method='leastsq')
                err_dict2, yerrdoubleGauss = self.bootstrap_err(x, self.doubleLineModel, y, yerr, params2, conv_args, num_boots=500, method='leastsq')

            

            for k in self.lineModel.param_names:
                params1[k] = [params1[k], err_dict1[k]]
            for k in self.doubleLineModel.param_names:
                params2[k] = [params2[k], err_dict2[k]]

            if i < 20 and self.verbose:
                plt.plot(x, y, label='emission line')
                plt.fill_between(x, y-yerr, y+yerr, alpha=0.2)
                plt.plot(x, ygauss, label='single conv')
                plt.fill_between(x, ygauss-yerrGauss, ygauss+yerrGauss, alpha=0.2)
                plt.legend()
                plt.show()

            self.df.at[i, 'BIC1'] = bic1
            self.df.at[i, 'BIC2'] = bic2
            self.df.at[i, 'params1']  = params1
            self.df.at[i, 'params2'] = params2
            self.df.at[i, 'conv_wavelength'] = xconv * self.wavelength_units
            self.df.at[i, 'gauss_flux'] = ygauss * self.flux_units
            self.df.at[i, 'gauss_fluxerr'] = yerrGauss * self.flux_units
            self.df.at[i, 'double_gauss_flux'] = y2gauss * self.flux_units
            self.df.at[i, 'double_gauss_fluxerr'] = y2errGauss * self.flux_units

            if i%10 == 0:
                with open("line_df.pkl", "wb") as f:
                    pkl.dump(self.df, f)


# Initialize spectrum class with HST files
The data is interesting, I don't have older data, and the cspec files are weird.

In [447]:
x1dsum_files = glob(os.path.join('HST', 'anonymous37222', '*x1dsum.fits'))
all_spectra = spectrum(x1dsum_files, linelist_file='HST/emission_lines.csv', dataDir='HST/', convolve=False)
all_spectra.coaddwave(all_spectra.spec_df, columns = ['obsdate', 'filter'])
all_spectra.H2lines_fromspectrum(all_spectra.spec_df)

,filename,obsdate,instrument,filter,dispfile,LSFfile,cenwave,wavelength,flux,fluxerr,segment,H2line_idx,"[nu', J']",labline,transition_prob
0,HST/anonymous37222/leit1f030_x1dsum.fits,2021-08-13,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1589_LP4_cn.dat,1589,"[1385.9931276253203 Angstrom, 1386.00536736705...","[0.0 erg / (Angstrom s cm2), 0.0 erg / (Angstr...","[0.0 erg / (Angstrom s cm2), 0.0 erg / (Angstr...","[[1574.1758709162705, 1774.8277413948108], [13...","[19173, 19752, 2397, 10087, 19635, 2634, 10457...","[[3,13], [3,13], [4,13], [4,13], [4,13], [3,16...","[1608.33, 1615.43, 1415.33, 1509.45, 1613.99, ...","[0.139, 0.125, 0.037, 0.023, 0.092, 0.05, 0.05..."
1,HST/anonymous37222/leit1c040_x1dsum.fits,2021-08-10,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.254196584923 Angstrom, 1421.266445181324...","[4.736898099985594e-15 erg / (Angstrom s cm2),...","[5.222956335908761e-15 erg / (Angstrom s cm2),...","[[1609.586167640663, 1810.3182110654668], [142...","[15273, 16330, 7200, 16095, 7571, 14043, 17259...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
2,HST/anonymous37222/leit1k040_x1dsum.fits,2021-08-18,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.250438441241 Angstrom, 1421.262687005296...","[8.536568923043122e-15 erg / (Angstrom s cm2),...","[6.194585743376239e-15 erg / (Angstrom s cm2),...","[[1609.5819599789054, 1810.3134733167517], [14...","[15274, 16331, 7201, 16096, 7571, 14043, 17260...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
3,HST/anonymous37222/leso2d010_x1dsum.fits,2022-06-16,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.3367712210866 Angstrom, 1421.34902052907...","[8.842805948548497e-16 erg / (Angstrom s cm2),...",[1.3567639658497993e-15 erg / (Angstrom s cm2)...,"[[1609.6796802860497, 1810.4233852465268], [14...","[15266, 16315, 7193, 16080, 7564, 14035, 17244...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
4,HST/anonymous37222/leit1h040_x1dsum.fits,2021-08-15,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.251682067404 Angstrom, 1421.263930642086...","[5.668530234803062e-15 erg / (Angstrom s cm2),...",[5.5346257574072025e-15 erg / (Angstrom s cm2)...,"[[1609.58333202459, 1810.3150195192381], [1421...","[15273, 16331, 7201, 16096, 7571, 14043, 17260...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
5,HST/anonymous37222/leph1d040_x1dsum.fits,2022-08-11,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.254063552068 Angstrom, 1421.266312147713...",[-2.957552869772527e-17 erg / (Angstrom s cm2)...,"[4.119783564383839e-15 erg / (Angstrom s cm2),...","[[1609.5861381699028, 1810.3181692097505], [14...","[15273, 16330, 7200, 16095, 7571, 14043, 17259...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
6,HST/anonymous37222/lephae040_x1dsum.fits,2022-08-24,COS,G160M,HST/2bj2256ml_disp.fits,HST/aa_LSFTable_G160M_1623_LP4_cn.dat,1623,"[1421.2488942629827 Angstrom, 1421.26114281391...","[9.00888619710672e-15 erg / (Angstrom s cm2), ...","[6.532031193179931e-15 erg / (Angstrom s cm2),...","[[1609.5802403789269, 1810.3115387103865], [14...","[15274, 16331, 7201, 16096, 7572, 14043, 17260...","[[3,13], [3,13], [4,13], [4,13], [3,16], [3,16...","[1608.33, 1615.43, 1509.45, 1613.99, 1513.99, ...","[0.139, 0.125, 0.023, 0.092, 0.057, 0.122, 0.0..."
7,HST/anonymous37222/leph1j030_x1dsum

# Some code to investigate fluxerr and flux in the spectrum and how it affects peak finding

In [448]:
date_filter = all_spectra.spec_df['obsdate']=='2022-08-14'

flux = all_spectra.spec_df.loc[date_filter, 'flux'].values[0].value
fluxerr = all_spectra.spec_df.loc[date_filter, 'fluxerr'].values[0].value
wavelength = all_spectra.spec_df.loc[date_filter, 'wavelength'].values[0].value

peaks, _ = find_peaks(flux, height=fluxerr*2, distance=50)
#print(peaks)
plt.plot(wavelength, flux)
plt.scatter(wavelength[peaks], flux[peaks], c='red', marker='x')
#plt.hlines(np.mean(fluxerr[flux>0]), min(wavelength), max(wavelength), 'r','--')
plt.plot(wavelength, fluxerr, lw=0.5, label='error')
plt.legend()
#plt.xlim(left=1390)
plt.show()

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 480x360 with 1 Axes>

In [449]:
for _, row in all_spectra.spec_df.iterrows():
    plt.plot(row['wavelength'], row['flux'])
    plt.scatter(row['wavelength'][row['H2line_idx']], row['flux'][row['H2line_idx']], c='r', marker='x')
    plt.title(row['obsdate']+' '+row['filter'])
    plt.xlabel(r'Wavelength ($\AA$)')
    plt.ylabel(r'Flux ($erg \AA^{-1} s^{-2} cm^{-2}$)')
    plt.show()

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 480x360 with 1 Axes>

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 480x360 with 1 Axes>

RuntimeError: Failed to process string with tex because dvipng could not be found

...

<Figure size 480x360 with 1 Axes>

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 480x360 with 1 Axes>

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 480x360 with 1 Axes>

# Compile properties of individual lines

In [450]:
all_spectra.spec_df = all_spectra.spec_df[all_spectra.spec_df["[nu', J']"].notna() & (all_spectra.spec_df["[nu', J']"].apply(len) > 0)] #remove the spectra (coadded for one observation date) that don't have any H2 lines
line_df = all_spectra.get_lines(all_spectra.spec_df) #rearrange df by progressions, not obsdate

# Look for contaminated lines
Transitions we have:
{'[0,3]', '[2,12]', '[0,2]', '[1,7]', '[4,13]', '[0,1]', '[3,16]', '[2,15]', '[1,4]', '[3,0]', '[4,4]', '[3,13]'}

In [451]:
exclude_lines = {
    "[0,1]": {
        "dates": ["2022-08-11", "2022-08-24", "2011-07-20", "2021-08-11", "2021-08-11", "2021-08-11", "2011-07-20", "2012-02-28", "2012-02-28", "2021-08-16", "2022-08-14", "2022-08-11", "2022-08-11"],
        "wavelengths": [1460.17, 1521.59, 1521.59, 1398.95, 1460.17, 1521.59, 1398.95, 1521.59, 1398.95, 1521.59, 1521.59, 1398.95, 1521.59]
    },
    "[0,2]": {
        "dates": ["2022-06-16", "2022-08-11", "2022-08-11", "2021-08-11", "2021-08-11", "2021-08-11", "2012-02-28", "2022-08-10", "2022-08-13", "All"],
        "wavelengths": [1525.15, 1463.83, 1525.15, 1402.65, 1463.83, 1525.15, 1402.65, 1525.15, 1525.15, 1525.15]
    },
    "[1,4]": {
        "dates": ["2022-08-11", "2021-08-11", "2021-08-11", "2021-08-11", "2021-08-11"],
        "wavelengths": [1431.01, 1431.01, 1446.12, 1489.57, 1504.76]
    },
    "[1,7]": {
        "dates": ["All", "All", "2022-08-11", "2022-08-11", "2011-07-20", "2021-08-11", "2021-08-11", "2021-08-11", "2021-08-11", "2012-02-28", "2021-08-21", "2022-08-18"],
        "wavelengths": [1556.87, 1524.65, 1467.08, 1500.45, 1556.87, 1467.08, 1500.45, 1524.65, 1556.87, 1556.87, 1556.87, 1556.87]
    },
    "[2,12]": {
        'dates': ['2021-08-10', '2021-08-18', '2021-08-18', '2022-06-16', '2022-06-16', '2022-08-1', '2022-08-11', '2022-08-11', '2021-08-20', '2011-07-20', '2021-08-11', '2021-08-11', '2021-08-11', '2012-02-28', '2021-08-16', '2021-08-12', '2022-08-10', "All", "2022-08-11", "All"],
        "wavelengths": [1453.10, 1555.89, 1588.8, 1453.1, 1555.89, 1453.1, 1555.89, 1588.8, 1555.89, 1588.8, 1453.1, 1555.89, 1588.8, 1555.89, 1453.1, 1555.89, 1453.1, 1434.54, 1453.1, 1453.1]
    },
    "[4,13]": {
        "dates": [
            "2011-07-20", "All", "2021-08-10", "2021-08-11", "2021-08-11",
            "2021-08-14", "2021-08-16", "2021-08-17", "2022-08-11", "2022-08-11",
            "2022-08-13", "2022-08-24", "2022-06-16", "All"
        ],
        "wavelengths": [
            1415.33, 1509.45, 1415.33, 1415.33, 1613.99,
            1415.33, 1415.33, 1415.33, 1415.33, 1613.99,
            1415.33, 1415.33, 1415.33, 1613.99
        ]
    },
    "[3,13]": {
        "dates": [
            "All", "2012-02-28", "2021-08-11", "2022-08-11", "2022-08-15",
            "2022-08-16", "2022-08-17", "2022-08-18", "2022-08-20", "2022-08-23",
            "2022-08-24", "2022-06-16"
        ],
        "wavelengths": [
            1615.43, 1608.33, 1608.33, 1608.33, 1608.33,
            1608.33, 1608.33, 1608.33, 1608.33, 1608.33,
            1608.33, 1608.33
        ]
    },
    "[4,4]": {
        "dates": [
            "All", "2021-08-10", "2021-08-10", "2021-08-11", "2021-08-11",
            "2021-08-15", "2021-08-21", "2022-08-11", "2022-08-11", "2022-08-15",
            "2021-08-10", "2021-08-11", "2021-08-15", "2021-08-17", "2021-08-18",
            "2022-08-11", "All"
        ],
        "wavelengths": [
            1477.05, 1526.55, 1613.72, 1526.55, 1613.72,
            1526.55, 1526.55, 1526.55, 1613.72, 1526.55,
            1477.05, 1477.05, 1477.05, 1477.05, 1477.05,
            1477.05, 1526.55
        ]
    }
}

In [452]:
# exclude bad/contaminated lines from coadding
bad_transitions = ['[3,0]', '[2,15]', '[3,16]', '[4,0]', '[0,3]', '[3,13]']
bad_transitions_mask = ~line_df['transition'].isin(bad_transitions)
line_df = line_df[bad_transitions_mask]

contaminated = []
for transition in exclude_lines.keys():
    dates = exclude_lines[transition]['dates']
    lines_ex = exclude_lines[transition]['wavelengths']
    for d, w in zip(dates, lines_ex):
        if d == "All":
            exclude_idx = line_df.index[(line_df['transition']==transition)&(line_df['labline']==w)].tolist()
        else:
            exclude_idx = line_df.index[(line_df['transition']==transition)&(line_df['labline']==w)&(line_df['obsdate']==d)].tolist()
        contaminated.extend(exclude_idx)

line_df = line_df.drop(index=contaminated)

line_df = line_df[line_df['obsdate']!='2021-08-11']
#Drop duplicates
line_df = line_df[~line_df.duplicated(['obsdate', 'transition', 'labline'])].reset_index(drop=True)

In [453]:
blue_transitions = {'[2,12]', '[4,13]', '[4,4]'}
mask = line_df['transition'].isin(blue_transitions)
blue_df = line_df[mask].copy()
all_spectra.coaddwave(blue_df, columns=['labline'])

In [454]:
bad_transitions = ['[2,12]', '[4,13]', '[4,4]']
bad_transitions_mask = ~line_df['transition'].isin(bad_transitions)
line_df = line_df[bad_transitions_mask]

# Model the emission lines and save the parameters

In [455]:
# labline: mask array
line_mask_dict = {
    1453.1:200,
    1555.89:[75,175],
    1463.83:200,
    1613.72:[75,200],
    1415.33:180 
}

In [456]:
#build dataframe for convolution information
def make_disp_df(files):
    disptable_info = []
    for file in files:
        disptable = fits.open(file, ignore_missing_simple=True)[1].data
        disptable = disptable[disptable['aperture']=="PSA"]
        cols = {}
        for col in ['SEGMENT', 'CENWAVE', 'OPT_ELEM','COEFF', 'D_TV03', 'D']:
            cols[col] = disptable[col]
        cols['disptab']=file
        disptable_info.append(cols)
    disp_df = pd.DataFrame(disptable_info).set_index('disptab')
    return disp_df
disp_df = make_disp_df(list(set(line_df['dispfile'])))
def make_lsf_df(files):
    lsf_info = []
    for file in files:
        # This is the table of all the LSFs: called "lsf"
        # The first column is a list of the wavelengths corresponding to the line profile, so we set our header accordingly
        hs = 0
        lsf = Table.read(file, format="ascii", header_start=hs)

        # This is the range of each LSF in pixels (for FUV from -160 to +160, inclusive)
        # middle pixel of the lsf is considered zero ; center is relative zero
        pix = np.arange(len(lsf)) - len(lsf) // 2  # integer division to yield whole pixels

        # the column names returned as integers.
        lsf_wvlns = np.array([int(float(k)) for k in lsf.keys()])
        lsf_info.append({'lsf_file':file, 'lsf':lsf, 'pix':pix, 'w':lsf_wvlns})
    lsf_df = pd.DataFrame(lsf_info).set_index('lsf_file')
    return lsf_df
lsf_df = make_lsf_df(list(set(line_df['LSFfile'])))


In [457]:
line_df_test = line_df.iloc[0:10].copy()

My parameter errors don't matter, because this is not the model I am using! since I am convolving the model, using the gaussian errors is no longer accurate to the actual model. Error estimation using the errors on the gaussian parameters should actually be by bootstrapping or smth!

In [469]:
fitting = ModelFitting(line_df, line_mask_dict, verbose=False)
blue_fitting = ModelFitting(blue_df, line_mask_dict)
fitting.fit_lines()
blue_fitting.fit_lines()

Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors
Bootstrapping errors


In [ ]:
for i, row in line_df_test.iterrows():

    flux = row['flux'].value
    fluxerr = row['fluxerr'].value
    wave = row['wavelength'].value
    
    ygauss = row['gauss_flux']
    ygausserr = row['gauss_fluxerr']

    y2gauss = row['double_gauss_flux']
    y2gausserr = row['double_gauss_fluxerr']

    waveconv = row['conv_wavelength'].value
    fig, ax = plt.subplots(1,2, figsize=(12,7))
    
    ax[0].plot(waveconv, ygauss, label='single conv gauss')
    ax[0].fill_between(waveconv, ygauss-ygausserr, ygauss+ygausserr, color='red', alpha=0.4,)

    ax[0].plot(wave, flux, '--', label='emission line')
    ax[0].fill_between(wave, flux-fluxerr, flux+fluxerr, color='black', alpha=0.2)
    ax[1].plot(wave, flux, '--', label='emission line')
    ax[1].fill_between(wave, flux-fluxerr, flux+fluxerr, color='black', alpha=0.2)

    ax[1].plot(waveconv, y2gauss, label='double conv gauss')
    ax[1].fill_between(waveconv, y2gauss-y2gausserr, y2gauss+y2gausserr, color='red', alpha=0.4,)
    
    ax[0].legend()
    ax[1].legend()
    plt.show()

# Coadd lines in velocity space

In [475]:
print(template1v, len(template1v))

[-340.69206001 -338.41674096 -336.14142191 -333.86610286 -331.59078381
 -329.31546476 -327.04014571 -324.76482666 -322.48950762 -320.21418857
 -317.93886952 -315.66355047 -313.38823142 -311.11291237 -308.83759332
 -306.56227427 -304.28695523 -302.01163618 -299.73631713 -297.46099808
 -295.18567903 -292.91035998 -290.63504093 -288.35972189 -286.08440284
 -283.80908379 -281.53376474 -279.25844569 -276.98312664 -274.70780759
 -272.43248854 -270.1571695  -267.88185045 -265.6065314  -263.33121235
 -261.0558933  -258.78057425 -256.5052552  -254.22993615 -251.95461711
 -249.67929806 -247.40397901 -245.12865996 -242.85334091 -240.57802186
 -238.30270281 -236.02738376 -233.75206472 -231.47674567 -229.20142662
 -226.92610757 -224.65078852 -222.37546947 -220.10015042 -217.82483138
 -215.54951233 -213.27419328 -210.99887423 -208.72355518 -206.44823613
 -204.17291708 -201.89759803 -199.62227899 -197.34695994 -195.07164089
 -192.79632184 -190.52100279 -188.24568374 -185.97036469 -183.69504564
 -181.

In [ ]:
#Calculate the velocity distribution fot every fit
blue_df['BIC1'] = pd.Series([None] * len(blue_df), dtype=object)
for i, row in blue_df.iterrows():
    wavelengths = row['conv_wavelength']
    labline = row['labline']
    velocity = all_spectra.radial_velocity(wavelengths, labline)
    blue_df.at[i, 'velocity'] = velocity

In [476]:
columns = ['transition', 'obsdate']
combinations = blue_df[columns].drop_duplicates()
blue_df['single_coadded_flux'] = pd.Series([None] * len(blue_df), dtype=object)
blue_df['single_coadded_fluxerr'] = pd.Series([None] * len(blue_df), dtype=object)
blue_df['double_coadded_flux'] = pd.Series([None] * len(blue_df), dtype=object)
blue_df['double_coadded_fluxerr'] = pd.Series([None] * len(blue_df), dtype=object)
drop_indices = []
for _, row in combinations.iterrows():
    unique_values = [row[col] for col in columns]
    # Filter the group
    mask = np.logical_and.reduce([(blue_df[col] == val) for col, val in zip(columns, unique_values)])
    group = blue_df[mask]
    
    coadded_singleflux = 0
    template1v = group['velocity'].iloc[0]
    coadd_singleerror = 0

    coadded_doubleflux = 0
    template2v = group['velocity'].iloc[0]
    coadd_doubleerror = 0
    
    for _, row in group.iterrows():
        #Coadd both Gaussians
        ygauss = row['gauss_flux']
        ygausserr = row['gauss_fluxerr']

        y2gauss = row['double_gauss_flux']
        y2gausserr = row['double_gauss_fluxerr']
        
        resampled_singleflux = np.interp(template1v, row['velocity'], ygauss)
        resampled_singlefluxerr = np.interp(template1v, row['velocity'], ygausserr)
        resampled_doubleflux = np.interp(template2v, row['velocity'], y2gauss)
        resampled_doublefluxerr = np.interp(template2v, row['velocity'], y2gausserr)
        coadded_singleflux += resampled_singleflux
        coadd_singleerror += (resampled_singlefluxerr)**2
        coadded_doubleflux += resampled_doubleflux
        coadd_doubleerror += (resampled_doublefluxerr)**2

    N = len(group)
    coadded_singleflux = coadded_singleflux/N
    coadded_singleerror = np.sqrt(coadd_singleerror)/np.sqrt(N)
    first_index = group.index[0]
    # All the first index properties are inherited and the rest is dropped
    blue_df.at[first_index, 'single_coadded_flux'] = coadded_singleflux*all_spectra.flux_units
    blue_df.at[first_index, 'single_coadded_fluxerr'] = coadd_singleerror*all_spectra.flux_units
    blue_df.at[first_index, 'double_coadded_flux'] = coadded_doubleflux*all_spectra.flux_units
    blue_df.at[first_index, 'double_coadded_fluxerr'] = coadd_doubleerror*all_spectra.flux_units
    drop_indices.extend(group.index[1:])

# df.drop(drop_indices, inplace=True)
# df.reset_index(inplace=True, drop=True)
#return prog_df
#Re-calculate 

ValueError: fp and xp are not of the same length.

In [ ]:
prog_df = line_df.copy()
all_spectra.coaddvel(prog_df, columns=['transition', 'obsdate'])

## Plot coadded lines

In [ ]:
#ibm_colors = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']
for transition_plot in ['[1,4]']:# prog_df['transition'].unique():
#for transition_plot in ['[1,7]']:
    mask = prog_df['transition']==transition_plot

    for _, row_prog in prog_df[mask].iterrows():
        fig, ax = plt.subplots(1,2, figsize=(7,3))
        fig.suptitle(row_prog['obsdate']+' '+transition_plot)
        coadd_lines = line_df[(line_df['transition']==transition_plot) & (line_df['obsdate']==row_prog['obsdate'])]
        for _, row_line in coadd_lines.iterrows():
            velocity = all_spectra.radial_velocity(row_line['wavelength'], row_line['labline'])
            ax[0].plot(velocity, row_line['flux'], alpha=0.4, label=row_line['labline'])
            ax[1].plot(velocity, row_line['flux']/row_line['fluxerr'], alpha=0.4, label=row_line['labline'])

            
        ax[0].plot(row_prog['velocity'], row_prog['flux'], c='black', ls='--', label='coadd')
        ax[0].vlines(0, 0,1.3e-13, 'r')

        ax[1].plot(row_prog['velocity'], row_prog['flux'].value/row_prog['fluxerr'].value, c='black', ls='--', label='coadd')
        
        ax[0].set_xlabel('Radial Velocity'+' '+u.format.Latex().to_string(row_prog['velocity'].unit, fraction=False))
        ax[1].set_xlabel('Radial Velocity'+' '+u.format.Latex().to_string(row_prog['velocity'].unit, fraction=False))
        
        ax[0].set_ylabel('Flux'+' '+u.format.Latex().to_string(row_prog['flux'].unit, fraction=False))
        ax[0].text(0.05, 0.95, f'Num lines = {len(coadd_lines)}', transform=plt.gca().transAxes, verticalalignment='top')
        ax[0].set_ylim(0, 3e-13)
        ax[0].set_xlim(-300,300)
        ax[0].legend(loc='lower right')
        ax[0].set_title('Coadded Spectral Line')
        ax[1].set_title('SNR')
        plt.show()

# Time-series plots of parameters and integrated fluxes

In [ ]:
def integrated_flux(fluxes):
    return np.sum(fluxes)

int_fluxes = prog_df['flux'].map(lambda x: np.sum(x)/all_spectra.flux_units)
for transition_plot in prog_df['transition'].unique():
    mask = prog_df['transition'] == transition_plot
    x = prog_df['obsdate'][mask]
    y = int_fluxes[mask]
    plt.plot(x, y)
    plt.ylabel('Integrated flux')
    plt.xlabel('Date')
    plt.title(transition_plot)
    plt.gca().tick_params(axis='x', labelrotation=60, labelsize=8)
    plt.show()

In [ ]:
def fitParamPlot(transition):
    dates = list(fit_dict[transition].keys())
    fig, axs = plt.subplots(1, len(fit_keys), figsize=(17,5), layout='tight')
    for i, param in enumerate(fit_keys):
        y = [fit_dict[transition][date][param][0] for date in dates]
        yerr = [fit_dict[transition][date][param][1] for date in dates]
        yerr = [np.nan if err==None else err for err in yerr]
        axs[i].errorbar(dates, y, yerr=yerr, fmt='o', marker='.', ecolor='gray')
        axs[i].set_title(param)
        axs[i].set_ylabel(param)
        axs[i].tick_params(axis='x', labelrotation=60, labelsize=8)
        if param == 'amp' or param == 'sig' or param == 'cen':
            axs[i].set_ylim(.3*min(y), 1.3*max(y))

    fig.suptitle(f'Parameters timeseries for {transition}')
    fig.supxlabel('Dates')
    fig.show()

for transition_plot in fit_dict.keys():
    fitParamPlot(transition_plot)

# Search for asymmetry by mirroring diff

Should reflect along the peak of flux

In [ ]:
cmap = mpl.pyplot.get_cmap('seismic', lut=len(prog_df['transition'].unique()))
clrs = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    clrs.append(mpl.colors.rgb2hex(rgba))
ordered_transitions = ['[3,13]', '[4,13]', '[4,4]', '[1,7]', '[1,4]', '[0,1]', '[0,2]', '[2,12]']
lya_colors = dict(zip(ordered_transitions, clrs))

for date in prog_df['obsdate'].unique():
    fig, ax = plt.subplots()
    offset = 1#e-13
    prog_df['transition'] = pd.Categorical(prog_df['transition'], categories=ordered_transitions, ordered=True)
    prog_from_date = prog_df[prog_df['obsdate']==date].sort_values(by='transition', ignore_index=True)
    for i, row_mirror in prog_from_date.iterrows():
        vel_arr = row_mirror['velocity']
        flux_arr = row_mirror['flux']
        flux_arr = flux_arr/np.max(flux_arr) #normalize the flux array
        mirrored_vel = 2 * vel_arr[np.argmax(flux_arr[125:175])] - vel_arr #mirror the velocity array along the maximum flux value found in the center of the plot
        sorted_indices = np.argsort(mirrored_vel)
        mirrored_flux = flux_arr.copy()
        x_mirrored = mirrored_vel[sorted_indices]
        y_mirrored = mirrored_flux[sorted_indices]
        residuals_x = vel_arr
        residuals_y = flux_arr - y_mirrored
        ax.plot(residuals_x, residuals_y.value - i*offset, label=row_mirror['transition'], c=lya_colors[row_mirror['transition']])
        ax.set_xlabel('Radial Velocity'+' ('+u.format.Latex().to_string(row_mirror['velocity'].unit, fraction=False)+')')
        ax.set_ylabel('Normalized Flux'+' ('+u.format.Latex().to_string(row_mirror['flux'].unit, fraction=False)+')')
        ax.set_title(row_mirror['obsdate'])
    ax.legend()
    plt.show()